In [1]:
import pickle

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup

import plotly.graph_objects as go

import pandas as pd

from datetime import datetime, timedelta

In [2]:
# All_fight_data = [Events] Ordered from old -> new.
    # Event = [Fights, Date, Venue]
        # Fight = [Fighter1, Fighter2, Result]
            # Fighter1, Fighter2 = [Name, Total, Strikes]
                # Totals = [Total, First, Second, Third, Fourth, Fifth]
                    # Total, First, etc = [Knockdowns, Takedowns, Reversals, Submission Attempted, Control Time]
                        # Takedowns = [Landed, Attempted]
                # Strikes = [Total, First, Second, Third, Fourth, Fifth]
                    # Total, First, etc = [Significant Strikes, Head, Body, Leg, Distance, Clinch, Ground]
                        # Everything = [Landed, Attempted]
            # Result = [Winner, Method, Round, Time, Referee, Weight, Bonus, Details]

In [64]:
save_data(all_f_data, 'all_f_data')
save_data(all_r_data, 'all_r_data')

In [3]:
def open_data(name):
    with open(name, "rb") as fp:
        return pickle.load(fp)

In [4]:
def save_data(data, name):
    with open(name, "wb") as fp:
        pickle.dump(data, fp)

In [5]:
def all_fighter_names(data):
    all_names = []
    for event in data:
        for fight in event[0]:
            if fight[0][0] not in all_names:
                all_names.append(fight[0][0])
                
            if fight[1][0] not in all_names:
                all_names.append(fight[1][0])
    return all_names

In [6]:
def all_referee_names(data):
    all_names = []
    for event in data:
        for fight in event[0]:
            if fight[2][4] not in all_names:
                all_names.append(fight[2][4])
    
    return all_names

In [7]:
def convert_date(date):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    month = date.split(' ')[0]
    month = months.index(month) + 1
    
    day = int(date.split(',')[0].split(' ')[1])
    year = int(date.split(', ')[1])
    date = datetime(year, month, day)
    return date

In [58]:
weight = ["Heavyweight", "Light Heavyweight", "Middleweight", "Welterweight", "Lightweight", "Featherweight", 
          "Bantamweight", "Flyweight", "Women's Bantamweight", "Women's Strawweight", "Women's Flyweight", 
          "Women's Featherweight", "Open Weight", "Super Heavyweight", "Catch Weight"]

def fighter_data(data, name):
#     ["Name", "Date", "Winlose score","KO/TKO", "Submission",
#      "Decision", "Finish score", "Wins", "Losses", "Draws/NCs"
#      "Knockdowns", "Takedowns Landed", "Takedowns Attempted", 
#      "Reversals", "Submission Attempted", "Control Time"]

    # count = [winlose, ko, submission, decision, finish score, wins, losses, draws, knockdowns, 
    #          takedowns landed, takedowns attempted, reversals, submission attempted, control time, weight]
    count = [0 for i in range(15)]
    count[13] = timedelta()
    count[14] = 'Unspecified'
    all_data = []
    
    for event in data:
        for fight in event[0]:
            winner = fight[2][0]

            fighters = [fight[0][0], fight[1][0]]
            if name in fighters:
                index = fighters.index(name)
                
                if fight[index][1] != []:
                    total = fight[index][1][0]

                    count[8] += total[0]
                    count[9] += total[1][0]
                    count[10] += total[1][0]
                    count[11] += total[2]
                    count[12] += total[3]
                    
                    for w in weight[::-1]:
                        if w in fight[2][5]:
                            count[14] = w
                            break
                            
                    if count[14] not in weight:
                        count[14] = 'Unspecified'

                    if total[4] != '--':
                        time = datetime.strptime(total[4], '%M:%S').time()
                    else:
                        time = datetime.strptime('00:00', '%M:%S').time()
                    
                    delta = timedelta(minutes=time.minute, seconds=time.second)
                    count[13] += delta

                if fight[winner][0] == name:
                    count[0] += 1
                    count[5] += 1

                    if fight[2][1] == 'KO/TKO':
                        count[1] += 1
                        count[4] += 1
                    elif fight[2][1] == 'Submission':
                        count[2] += 1
                        count[4] += 1
                    else:
                        count[3] += 1
                        count[4] -= 1

                elif winner == 2:
                    count[7] += 1
                else:
                    count[0] -= 1
                    count[6] += 1
                
                data = [name, convert_date(event[1])] + count
                all_data.append(data)
    
    return all_data

In [62]:
def referee_data(data, name):
    # count = [ko, submission, decision, finish score, weight]
    count = [0 for i in range(5)]
    count[4] = 'Unspecified'
    all_data = []
    
    for event in data:
        for fight in event[0]:
            if name == fight[2][4]:
                
                for w in weight[::-1]:
                    if w in fight[2][5]:
                        count[4] = w
                        break

                if count[4] not in weight:
                    count[4] = 'Unspecified'
                    
                if fight[2][1] == 'KO/TKO':
                    count[0] += 1
                    count[3] += 1
                elif fight[2][1] == 'Submission':
                    count[1] += 1
                    count[3] += 1
                else:
                    count[2] += 1
                    count[3] -= 1
            
                data = [name, convert_date(event[1])] + count
                all_data.append(data)
    
    return all_data

In [63]:
all_fight_data = open_data('all_fight_data')
all_f_names = all_fighter_names(all_fight_data)
all_f_data = [fighter_data(all_fight_data, name) for name in all_f_names]
all_f_data = [item for sublist in all_f_data for item in sublist]

all_r_names = all_referee_names(all_fight_data)
all_r_data = [referee_data(all_fight_data, name) for name in all_r_names]
all_r_data = [item for sublist in all_r_data for item in sublist]

In [14]:
structure_r = ["Name", "Date", "KO/TKO", "Submission", "Decision", "Finish score"]
structure_f = ["Name", "Date", "Winlose score","KO/TKO", "Submission", "Decision", "Finish score", "Wins", "Losses", "Draws/NCs",
			   "Knockdowns", "Takedowns Landed", "Takedowns Attempted", "Reversals", "Submission Attempted", "Control Time", "Weight"]

In [15]:
df_r = pd.DataFrame(all_r_data, columns=structure_r)
names = ['Mark Smith', 'Herb Dean']
df_r = df_r[df_r['Name'].isin(names)]

df_r

,Name,Date,KO/TKO,Submission,Decision,Finish score
1340,Herb Dean,2004-04-02,0,0,1,-1
1341,Herb Dean,2004-04-02,1,0,1,0
1342,Herb Dean,2004-06-19,1,1,1,1
1343,Herb Dean,2004-06-19,1,2,1,2
1344,Herb Dean,2004-08-21,2,2,1,3
...,...,...,...,...,...,...
6124,Mark Smith,2022-12-10,93,43,141,-5
6125,Mark Smith,2022-12-10,93,44,141,-4
6126,Mark Smith,2022-12-17,93,44,142,-5
6127,Mark Smith,2022-12-17,94,44,142,-4


In [27]:
import plotly.express as px

df_f = pd.DataFrame(all_f_data, columns=structure_f)
names = ['Jon Jones ']
df_f = df_f[df_f['Name'].isin(names)]

fig = px.line(df_f, x="Date", y="Control Time", color="Name")
fig.update_layout(showlegend=False)

y
